In [ ]:
!pip install xgboost
conda install -c conda-forge s3fs
import s3fs

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [3]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/alllabs1000adm.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata = pd.read_csv(obj['Body'])
rawdata['itemid'] = rawdata['itemid'].astype("int64")
rawdata['hadm_id'] = rawdata['hadm_id'].astype("int64")

In [4]:
rawdata.shape

(64914520, 6)

In [4]:
unique_hadm_by_year = rawdata.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)

split year is 2178


In [5]:
hadm_id_indices = rawdata.groupby('hadm_id').apply(lambda group: {'start_index': group.index.min(), 'end_index': group.index.max()+1}).to_dict()
Labitemlist = rawdata['itemid'].value_counts().sort_values(ascending=False)
Labitemlist = Labitemlist.reset_index()
Labitemlist.columns = ['unique_value', 'count']
len(Labitemlist)

375

In [6]:
itemid = Labitemlist['unique_value'][2]
print(itemid)
this_rl = rawdata[rawdata['itemid']==itemid][['hadm_id','chartyear','valuenum','charttime_diff_hours']]
this_rl.shape

50902


(2042431, 4)

In [7]:
selected_rl = pd.DataFrame(columns=range(len(Labitemlist)*2+4), index=range(len(this_rl)))
i = 4
for name in Labitemlist['unique_value']:
    selected_rl.rename(columns={i: name}, inplace=True)
    i=i+1
    selected_rl.rename(columns={i: f"{name}_dt"}, inplace=True)
    i=i+1
selected_rl.rename(columns={0: 'hadm_id'}, inplace=True)
selected_rl.rename(columns={1: 'chartyear'}, inplace=True)
selected_rl.rename(columns={2: 'valuenum'}, inplace=True)
selected_rl.rename(columns={3: 'charttime_diff_hours'}, inplace=True)
selected_rl = selected_rl.set_index(this_rl.index)
selected_rl['hadm_id'] = this_rl['hadm_id']
selected_rl['chartyear'] = this_rl['chartyear']
selected_rl['valuenum'] = this_rl['valuenum']
selected_rl['charttime_diff_hours'] = this_rl['charttime_diff_hours']
selected_rl.shape

(2042431, 754)

In [ ]:
for inx, row in selected_rl.iterrows():
  hadm_id = row['hadm_id']
  start_index = hadm_id_indices.get(hadm_id)['start_index']
  # Get the end index for selection
  end_index = hadm_id_indices.get(hadm_id)['end_index']
  filtered_df = pd.concat([rawdata.iloc[start_index:inx], rawdata.iloc[inx + 1:end_index]])
  filtered_df['abs_time_diff'] = abs(filtered_df['charttime_diff_hours']-row['charttime_diff_hours'])
  idx_min_abs_time_diff = filtered_df.groupby('itemid')['abs_time_diff'].idxmin()
  pred_df = filtered_df.loc[idx_min_abs_time_diff, ['itemid', 'valuenum','abs_time_diff']]
  pred_df['itemid'] = pred_df['itemid'].astype("int64")
  for _,prow in pred_df.iterrows():
    iid = prow['itemid'].astype("int64")
    selected_rl.at[inx,iid] = prow['valuenum']
    selected_rl.at[inx,f"{iid}_dt"] = prow['abs_time_diff']

In [ ]:
selected_rl.head()

In [ ]:
file_name = f"alllabs1000adm_{itemid}.csv"
selected_rl.to_csv(f"alllabs1000adm_{itemid}.csv")
file_key = f"/Labrado/alllabs1000adm/alllabs1000adm_{itemid}.csv"
s3.upload_file(file_name, bucket_name,file_key)